In [1]:
# import modules from other folders
# source: https://stackoverflow.com/questions/49264194/import-py-file-in-another-directory-in-jupyter-notebook
# author: adhg (accessed 15 June 2025)
import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..', 'src/python'))
# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

from train_logistic_model import *
from lexicon import create_lexicon
import config as cfg

In [2]:
# computationally expensive
# prefer loading dataset when possibile

try:
    df_train = pd.read_csv(cfg.DATAFRAME_SAVE_PATH / "df_train.csv")
    df_val = pd.read_csv(cfg.DATAFRAME_SAVE_PATH / "df_val.csv")
    df_test = pd.read_csv(cfg.DATAFRAME_SAVE_PATH / "df_test.csv")
except FileNotFoundError:
    df_train, df_val, df_test = create_data_splits_with_kb_features(merge_train_test())

#### Train baseline model

In [ ]:
# generate lexicon on train data
create_lexicon(df_train)

In [3]:
X_train, y_train = split_feature_target(df_train, "label")
X_val, y_val = split_feature_target(df_val, "label")

In [ ]:
pipeline = create_pipeline()
pipeline.fit(X_train, y_train)

#### Valuate baseline model

In [ ]:
y_val_pred = pipeline.predict(X_val)
y_val_proba = pipeline.predict_proba(X_val)[:,1]

from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
print("Acc val:", accuracy_score(y_val, y_val_pred))
print("AUC val:", roc_auc_score(y_val, y_val_proba))
print(classification_report(y_val, y_val_pred))